<a href="https://colab.research.google.com/github/madhurapi/Recommender-System/blob/main/4_Recommender_System_With_Matrix_Factorization_%26_KNN_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries 

In [ ]:
import pandas as pd
import numpy as np

### Import the movies dataset

In [ ]:
movie_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommender System/movies.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
rating_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommender System/ratings.csv')

### Checking the tables 

In [ ]:
movie_df.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
rating_df.head(10)

,userId,movieId,rating,timestamp
0,1,16,4.0,1217897793
1,1,24,1.5,1217895807
2,1,32,4.0,1217896246
3,1,47,4.0,1217896556
4,1,50,4.0,1217896523
5,1,110,4.0,1217896150
6,1,150,3.0,1217895940
7,1,161,4.0,1217897864
8,1,165,3.0,1217897135
9,1,204,0.5,1217895786


### Now combine the two tables and drop things we dont have to use

In [ ]:
combine_movie_rating = pd.merge(rating_df, movie_df, on='movieId')
combine_movie_rating.head(10)

,userId,movieId,rating,timestamp,title,genres
0,1,16,4.0,1217897793,Casino (1995),Crime|Drama
1,9,16,4.0,842686699,Casino (1995),Crime|Drama
2,12,16,1.5,1144396284,Casino (1995),Crime|Drama
3,24,16,4.0,963468757,Casino (1995),Crime|Drama
4,29,16,3.0,836820223,Casino (1995),Crime|Drama
5,31,16,4.0,832397760,Casino (1995),Crime|Drama
6,47,16,4.0,853361808,Casino (1995),Crime|Drama
7,60,16,4.0,1162941904,Casino (1995),Crime|Drama
8,62,16,4.5,1267576909,Casino (1995),Crime|Drama
9,88,16,3.5,1180879072,Casino (1995),Crime|Drama


In [ ]:
columns = ['timestamp', 'genres']
combine_movie_rating = combine_movie_rating.drop(columns, axis=1)
combine_movie_rating.head(10)

,userId,movieId,rating,title
0,1,16,4.0,Casino (1995)
1,9,16,4.0,Casino (1995)
2,12,16,1.5,Casino (1995)
3,24,16,4.0,Casino (1995)
4,29,16,3.0,Casino (1995)
5,31,16,4.0,Casino (1995)
6,47,16,4.0,Casino (1995)
7,60,16,4.0,Casino (1995)
8,62,16,4.5,Casino (1995)
9,88,16,3.5,Casino (1995)


In [ ]:
combine_movie_rating = combine_movie_rating.dropna(axis = 0, subset = ['title'])

movie_ratingCount = (combine_movie_rating.
     groupby(by = ['title'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['title', 'totalRatingCount']]
    )
movie_ratingCount.head(10)

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),1
3,'Til There Was You (1997),3
4,"'burbs, The (1989)",20
5,'night Mother (1986),1
6,(500) Days of Summer (2009),37
7,*batteries not included (1987),11
8,...And Justice for All (1979),10
9,10 (1979),3


In [ ]:
rating_with_totalRatingCount = combine_movie_rating.merge(movie_ratingCount, left_on = 'title', right_on = 'title', how = 'left')
rating_with_totalRatingCount.head(10)

,userId,movieId,rating,title,totalRatingCount
0,1,16,4.0,Casino (1995),84
1,9,16,4.0,Casino (1995),84
2,12,16,1.5,Casino (1995),84
3,24,16,4.0,Casino (1995),84
4,29,16,3.0,Casino (1995),84
5,31,16,4.0,Casino (1995),84
6,47,16,4.0,Casino (1995),84
7,60,16,4.0,Casino (1995),84
8,62,16,4.5,Casino (1995),84
9,88,16,3.5,Casino (1995),84


### Now drop the duplicate data

In [ ]:
user_rating = rating_with_totalRatingCount.drop_duplicates(['userId','title'])
user_rating.head(10)

,userId,movieId,rating,title,totalRatingCount
0,1,16,4.0,Casino (1995),84
1,9,16,4.0,Casino (1995),84
2,12,16,1.5,Casino (1995),84
3,24,16,4.0,Casino (1995),84
4,29,16,3.0,Casino (1995),84
5,31,16,4.0,Casino (1995),84
6,47,16,4.0,Casino (1995),84
7,60,16,4.0,Casino (1995),84
8,62,16,4.5,Casino (1995),84
9,88,16,3.5,Casino (1995),84


## Matrix Factorization

### Now create a matrix and fill 0 values  

In [ ]:
movie_user_rating_pivot = user_rating.pivot(index = 'userId', columns = 'title', values = 'rating').fillna(0)
movie_user_rating_pivot.head(10)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 (1979),...,[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),a/k/a Tommy Chong (2005),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X = movie_user_rating_pivot.values.T
X.shape

(10323, 668)

### Now lets fit the model

In [ ]:
import sklearn
from sklearn.decomposition import TruncatedSVD

SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

(10323, 12)

In [ ]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)
corr = np.corrcoef(matrix)
corr.shape

(10323, 10323)

### Now lets check the results

In [ ]:
movie_title = movie_user_rating_pivot.columns
movie_title_list = list(movie_title)
coffey_hands = movie_title_list.index("Guardians of the Galaxy (2014)")

In [ ]:
corr_coffey_hands  = corr[coffey_hands]
list(movie_title[(corr_coffey_hands >= 0.9)])

['Avengers, The (2012)',
 'Big Hero 6 (2014)',
 'Captain America: The Winter Soldier (2014)',
 'Cowboy Bebop (1998)',
 'Dark Knight Rises, The (2012)',
 'Dredd (2012)',
 'Edge of Tomorrow (2014)',
 'Ex Machina (2015)',
 'Guardians of the Galaxy (2014)',
 'Her (2013)',
 'How to Train Your Dragon 2 (2014)',
 'Looper (2012)',
 'Mad Max: Fury Road (2015)',
 'Nightcrawler (2014)',
 'Star Trek Into Darkness (2013)',
 'Thor: The Dark World (2013)',
 'X-Men: Days of Future Past (2014)']

In [ ]:
coffey_hands_1 = movie_title_list.index("xXx (2002)")

In [ ]:
corr_coffey_hands_1  = corr[coffey_hands_1]
list(movie_title[(corr_coffey_hands_1 >= 0.9)])

['16 Blocks (2006)',
 'Bad Company (2002)',
 'Blade: Trinity (2004)',
 'Captain America: The First Avenger (2011)',
 'Chronicles of Riddick, The (2004)',
 'Constantine (2005)',
 'Dark Shadows (2012)',
 'Eastern Promises (2007)',
 'Expendables, The (2010)',
 'Fast and the Furious, The (2001)',
 'Gone Baby Gone (2007)',
 'Grey, The (2012)',
 'Incredible Hulk, The (2008)',
 'Island, The (2005)',
 'King Kong (2005)',
 'Kingdom of Heaven (2005)',
 'Live Free or Die Hard (2007)',
 'Madagascar: Escape 2 Africa (2008)',
 'Man on Fire (2004)',
 'Men Who Stare at Goats, The (2009)',
 'Mission: Impossible III (2006)',
 'National Treasure (2004)',
 "Ocean's Twelve (2004)",
 'Pelican Brief, The (1993)',
 "Pirates of the Caribbean: At World's End (2007)",
 'Punisher, The (2004)',
 'Quantum of Solace (2008)',
 'Red (2010)',
 'Salt (2010)',
 'Sherlock Holmes: A Game of Shadows (2011)',
 'Shrek the Third (2007)',
 'Spider-Man 3 (2007)',
 'Stardust (2007)',
 'Tears of the Sun (2003)',
 'Thunderball (196

# KNN Algorithm

In [ ]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [ ]:
X_matrix = csr_matrix(X)

In [ ]:
knn = NearestNeighbors(n_neighbors=10,algorithm= 'brute', metric= 'cosine')
model_knn = knn.fit(X_matrix)

In [ ]:
# # Predict
# query_index = np.random.choice(X.shape[0])
# distance, indice = model_knn.kneighbors(X.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)
# for i in range(0, len(distance.flatten())):
#     if  i == 0:
#         print('Recmmendation for {0}:\n'.format(X.index[query_index]))
#     else:
#         print('{0}: {1} with distance: {2}'.format(i,X.index[indice.flatten()[i]],distance.flatten()[i]))

In [ ]:
%pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow import keras

In [ ]:
import os
import math
from zipfile import ZipFile
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import StringLookup

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(X, test_size=0.2, random_state=42)

In [ ]:
# converting the training and testing data into torch tensors

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.models import Model

movie_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(movie_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

prod = Dot(name="Dot-Product", axes=1)([book_vec, user_vec])
model = Model([user_input, book_input], prod)
model.compile('adam', 'mean_squared_error')

movie = Embedding()

NameError: ignored